### Initial setup

In [2]:
import cv2
import os
import json
from utils import *

images_dir = './samples'
input = 'input.json'
output = 'output.json'

### Check image type

In [8]:
keypoints_data = [
    82, 55, 39, 20, 8, 16, 30, 63, 28, 17, 31, 206, 307, 319, 302, 162, 217, 451, 154, 
    310, 293, 1354, 1360, 187, 149, 194, 130, 338, 214, 250, 166, 248, 165, 154, 295, 
    344, 168, 2413, 504, 287, 829, 187, 137, 99, 90, 128, 87, 78, 64, 131
]

for idx, value in enumerate(keypoints_data):
    print(f"Index: {idx}, Value: {value}")

Index: 0, Value: 82
Index: 1, Value: 55
Index: 2, Value: 39
Index: 3, Value: 20
Index: 4, Value: 8
Index: 5, Value: 16
Index: 6, Value: 30
Index: 7, Value: 63
Index: 8, Value: 28
Index: 9, Value: 17
Index: 10, Value: 31
Index: 11, Value: 206
Index: 12, Value: 307
Index: 13, Value: 319
Index: 14, Value: 302
Index: 15, Value: 162
Index: 16, Value: 217
Index: 17, Value: 451
Index: 18, Value: 154
Index: 19, Value: 310
Index: 20, Value: 293
Index: 21, Value: 1354
Index: 22, Value: 1360
Index: 23, Value: 187
Index: 24, Value: 149
Index: 25, Value: 194
Index: 26, Value: 130
Index: 27, Value: 338
Index: 28, Value: 214
Index: 29, Value: 250
Index: 30, Value: 166
Index: 31, Value: 248
Index: 32, Value: 165
Index: 33, Value: 154
Index: 34, Value: 295
Index: 35, Value: 344
Index: 36, Value: 168
Index: 37, Value: 2413
Index: 38, Value: 504
Index: 39, Value: 287
Index: 40, Value: 829
Index: 41, Value: 187
Index: 42, Value: 137
Index: 43, Value: 99
Index: 44, Value: 90
Index: 45, Value: 128
Index: 46

In [ ]:
with open(input, 'r') as json_file:
    data = json.load(json_file)

image_paths = data['image_files']
image_index = 37

image = cv2.imread(os.path.join(images_dir, image_paths[image_index]))
image = cv2.resize(image, (0, 0), fx = 0.15, fy = 0.15)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

complexity, num_keypoints = get_image_complexity(gray)
print(f'Complexity: {complexity} ({num_keypoints} features)')

cv2.imshow('Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

### GrabCut testing

In [ ]:
# ----------------------------------------------

with open(input, 'r') as json_file:
    data = json.load(json_file)

image_paths = data['image_files']
image_index = 11

image = cv2.imread(os.path.join(images_dir, image_paths[image_index]))
image = cv2.resize(image, (0, 0), fx = 0.15, fy = 0.15)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

complexity, _ = get_image_complexity(gray)

# ----------------------------------------------

blur_kernel = (9, 9) if complexity == 'Complex' else (7, 7)
bb_size = 100 if complexity == 'Complex' else 50
iterations = 5 if complexity == 'Complex' else 1
grabcut_mask = grab_cut(image, blur_kernel, bb_size, iterations)
grabcut = cv2.bitwise_and(image, image, mask=grabcut_mask)

# ----------------------------------------------

cv2.imshow('Image', image)
cv2.imshow('Grabcut', grabcut)
cv2.waitKey(0)
cv2.destroyAllWindows()

# ----------------------------------------------

### Background median testing

In [ ]:
# ----------------------------------------------

with open(input, 'r') as json_file:
    data = json.load(json_file)

image_paths = data['image_files']
image_index = 21

image = cv2.imread(os.path.join(images_dir, image_paths[image_index]))
image = cv2.resize(image, (0, 0), fx = 0.15, fy = 0.15)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

complexity, _ = get_image_complexity(gray)

# ----------------------------------------------

def process_image():
    complexity, _ = get_image_complexity(gray)
    blur_kernel = (5, 5)
    if complexity == 2:
        blur_kernel = (9, 9)
    elif complexity == 3:
        blur_kernel = (11, 11)
    bg_median_mask = background_median(hsv, hue_tolerance, saturation_tolerance, value_tolerance, blur_kernel)
    bg_median = cv2.bitwise_and(image, image, mask=bg_median_mask)
    contours, _ = cv2.findContours(bg_median_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = filter_contours(contours)
    image_copy = image.copy()
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(image_copy, (x, y), (x + w, y + h), (255, 255, 0), 2)
    combined_image = np.hstack((bg_median, image_copy))
    cv2.imshow(window_name, combined_image)

def update_hue_tolerance(*args):
    global hue_tolerance
    hue_tolerance = cv2.getTrackbarPos("Hue", window_name)
    process_image()

def update_saturation_tolerance(*args):
    global saturation_tolerance
    saturation_tolerance = cv2.getTrackbarPos("Saturation", window_name)
    process_image()

def update_value_tolerance(*args):
    global value_tolerance
    value_tolerance = cv2.getTrackbarPos("Value", window_name)
    process_image()

# ----------------------------------------------

hue_tolerance = 100
saturation_tolerance = 100
value_tolerance = 100

window_name = "TEST"
cv2.namedWindow(window_name)

cv2.createTrackbar("Hue", window_name, hue_tolerance, 180, update_hue_tolerance)
cv2.createTrackbar("Saturation", window_name, saturation_tolerance, 255, update_saturation_tolerance)
cv2.createTrackbar("Value", window_name, value_tolerance, 255, update_value_tolerance)

process_image()

cv2.waitKey(0)
cv2.destroyAllWindows()

# ----------------------------------------------

### Canny testing

In [13]:
# ----------------------------------------------

with open(input, 'r') as json_file:
    data = json.load(json_file)

image_paths = data['image_files']
image_index = 37

image = cv2.imread(os.path.join(images_dir, image_paths[image_index]))
image = cv2.resize(image, (0, 0), fx = 0.15, fy = 0.15)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

complexity, _ = get_image_complexity(gray)

# ----------------------------------------------

def process_image():
    complexity, _ = get_image_complexity(gray)
    blur_kernel = (5, 5)
    if complexity == 2:
        blur_kernel = (9, 9)
    elif complexity == 3:
        blur_kernel = (7, 7)

    canny_mask = canny_edges(gray, blur_kernel, low_threshold, high_threshold)
    canny = cv2.bitwise_and(image, image, mask=canny_mask)
    contours, _ = cv2.findContours(canny_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = filter_contours(contours)
    image_copy = image.copy()
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(image_copy, (x, y), (x + w, y + h), (255, 255, 0), 2)
    combined_image = np.hstack((canny, image_copy))
    cv2.imshow(window_name, combined_image)

def update_low_threshold(*args):
    global low_threshold
    low_threshold = cv2.getTrackbarPos("Low", window_name)
    process_image()

def update_high_threshold(*args):
    global high_threshold
    high_threshold = cv2.getTrackbarPos("High", window_name)
    process_image()

# ----------------------------------------------

low_threshold = 100
high_threshold = 200

window_name = "TEST"
cv2.namedWindow(window_name, cv2.WINDOW_AUTOSIZE)

cv2.createTrackbar("Low", window_name, low_threshold, 255, update_low_threshold)
cv2.createTrackbar("High", window_name, high_threshold, 255, update_high_threshold)

process_image()

cv2.waitKey(0)
cv2.destroyAllWindows()

# ----------------------------------------------